In [4]:
pip install xgboost pandas pyarrow fastparquet

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 27.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.3 MB/s eta 0:00:00:00:01
DEPRECATION: devscripts 2.22.1ubuntu1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of devscripts or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# SHARP RATIO Calculation
Replace the below code under utils/evalution_criteria -> calculate_sharpe_ratio()


In [3]:
def calculate_sharpe_ratio():
    try:
        #calculating sharpe:
        chars = pd.read_parquet("data/ctff_chars.parquet")
        # print(chars.head())
        pf = pd.read_csv("data/training_results.csv", names=["id", "eom", "w"], header=0)
        # print(pf.head())
        #getting rets attached to weights
        sharpe_df = pd.merge(
            pf.assign(
                id=pf['id'].astype('int64'),
                eom=pd.to_datetime(pf['eom']),
                w=pf['w'].astype('float64')
            )[['id', 'eom', 'w']],
            chars.assign(
                id=chars['id'].astype('int64'),
                eom=pd.to_datetime(chars['eom']),
                ret_exc_lead1m=chars['ret_exc_lead1m'].astype('float64')
            )[['id', 'eom', 'ret_exc_lead1m']],
            on=['id', 'eom'],
            how='left'
        )

        # Calculate weighted returns
        sharpe_df['w_ret'] = sharpe_df['w'] * sharpe_df['ret_exc_lead1m']
        print(sharpe_df['ret_exc_lead1m'].head())
        # Group by eom and calculate the sum of products
        rets = sharpe_df.groupby('eom')['w_ret'].sum()
        print(rets)
        # Calculate average and volatility
        average_ret = rets.mean()
        volatility = rets.std()

        # calculate sharpe
        sharpe = average_ret/volatility
        print(sharpe)
        return True, str(sharpe)

    except Exception as e:
        return False, str(e)
    

    
calculate_sharpe_ratio()

(False, "[Errno 2] No such file or directory: 'data/ctff_chars.parquet'")

In [ ]:
if []:
    print("hello")